<a href="https://colab.research.google.com/github/CloudScope/computer-vison-mlops/blob/main/modelTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initializing Nviidia GPU

In [ ]:
!nvidia-smi

Thu Jan 15 18:54:02 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Install Dependencies & Mount Drive

In [1]:
# Install Ultralytics for YOLO
!pip install ultralytics

# Mount Google Drive so you don't lose your work when the session ends
from google.colab import drive

# Check if GPU is working
import torch
print(f"Using GPU: {torch.cuda.get_device_name(0)}" if torch.cuda.is_available() else "GPU not found. Go to Runtime > Change runtime type")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.3 MB/s eta 0:00:00
Using GPU: Tesla T4


Create Project Folders

In [ ]:
import os

# Create folders
os.makedirs('/content/dataset/images/train', exist_ok=True)
os.makedirs('/content/dataset/labels/train', exist_ok=True)

print("Folders created. Now upload your images to the 'raw_images' folder in the left sidebar.")

Folders created. Now upload your images to the 'raw_images' folder in the left sidebar.


The Auto-Labeling Script

In [ ]:
import os
import cv2
from ultralytics import YOLO

# Load a specialized Motorcycle Helmet model directly from Hugging Face
# This model is better for riders than a construction 'hard hat' model
labeler = YOLO('/content/traffic.pt')

def auto_label_data():
    raw_path = '/content/drive/MyDrive/traffic/today_images'
    output_img_path = '/content/dataset/images/train'
    output_lbl_path = '/content/dataset/labels/train'

    img_list = [f for f in os.listdir(raw_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if len(img_list) == 0:
        print("❌ Error: No images found in /content/drive/MyDrive/traffic/today_images Upload some images first!")
        return

    print(f"Starting auto-labeling for {len(img_list)} images...")

    for img_name in img_list:
        img_path = os.path.join(raw_path, img_name)
        # Use a lower confidence (0.3) for auto-labeling to capture more potential helmets
        results = labeler(img_path, conf=0.3, verbose=False)[0]

        # Save image to dataset folder
        img = cv2.imread(img_path)
        cv2.imwrite(os.path.join(output_img_path, img_name), img)

        # Create YOLO label file
        label_file = os.path.join(output_lbl_path, img_name.rsplit('.', 1)[0] + '.txt')
        with open(label_file, 'w') as f:
            for box in results.boxes:
                # Class mapping: ensure it matches your expected [helmet, no-helmet]
                # Most safety models: 0 = Helmet, 1 = No-Helmet
                cls = int(box.cls[0])
                coords = box.xywhn[0].tolist() # Normalized [center_x, center_y, width, height]
                f.write(f"{cls} {coords[0]:.6f} {coords[1]:.6f} {coords[2]:.6f} {coords[3]:.6f}\n")

    print(f"✅ Success! Labels generated in {output_lbl_path}")

auto_label_data()

Starting auto-labeling for 41 images...
✅ Success! Labels generated in /content/dataset/labels/train


Verify the image lebling, this is not required in real time

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
import random

def visualize_labels(num_samples=3):
    img_dir = '/content/dataset/images/train'
    lbl_dir = '/content/dataset/labels/train'

    # Class names (should match your data.yaml)
    class_names = ['Helmet', 'PersonWithMotorcycle']
    colors = [(0, 255, 0), (0, 0, 255)] # Green for Helmet, Red for No-Helmet

    images = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    samples = random.sample(images, min(num_samples, len(images)))

    for img_name in samples:
        img_path = os.path.join(img_dir, img_name)
        lbl_path = os.path.join(lbl_dir, img_name.rsplit('.', 1)[0] + '.txt')

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w, _ = img.shape

        if os.path.exists(lbl_path):
            with open(lbl_path, 'r') as f:
                for line in f:
                    cls, cx, cy, bw, bh = map(float, line.split())

                    # Convert normalized YOLO coordinates to pixel coordinates
                    x1 = int((cx - bw/2) * w)
                    y1 = int((cy - bh/2) * h)
                    x2 = int((cx + bw/2) * w)
                    y2 = int((cy + bh/2) * h)

                    # Draw rectangle and label
                    cv2.rectangle(img, (x1, y1), (x2, y2), colors[int(cls)], 2)
                    cv2.putText(img, class_names[int(cls)], (x1, y1-10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[int(cls)], 2)

        plt.figure(figsize=(10, 10))
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Visualizing: {img_name}")
        plt.show()

visualize_labels(num_samples=44)

Create the data.yaml

In [ ]:
import yaml

data_config = {
    'path': '/content/dataset',
    'train': 'images/train',
    'val': 'images/train', # In Colab, we can use same for val to speed things up
    'nc': 2,
    'names': ['helmet', 'PersonWithMotorcycle']
}

with open('/content/dataset/data.yaml', 'w') as f:
    yaml.dump(data_config, f)

Train the Best Model

In [ ]:
from ultralytics import YOLO

# Load base YOLOv8 medium model
model = YOLO('yolov8m.pt')

# Start training
model.train(
    data='/content/dataset/data.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    device=0, # Use the T4 GPU
    name='helmet_project'
)

Ultralytics 8.4.2 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=helmet_project2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78a56c0d7950>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

Get Classes of a Model

In [ ]:
from ultralytics import YOLO

# 1. Load the entire model
model = YOLO('/content/traffic.pt')
print(model.names)


{0: 'Helmet', 1: 'PersonWithMotorcycle'}
